#**App Praevidens**

In [ ]:
!pip install --quiet --ignore-installed blinker streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 1.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 88.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.2/731.2 kB 40.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.6/207.6 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 71.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 66.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 72.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.1/321.1 kB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.

In [ ]:
!pip install -q pyngrok

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import shutil

ruta_base = '/content/drive/MyDrive/Colab Notebooks/'

shutil.copy(ruta_base + 'Modelo_RFR.pkl', '/content/')
shutil.copy(ruta_base + 'Modelo_RFC.pkl', '/content/')
shutil.copy(ruta_base + 'Label_Encoder_RFR.pkl', '/content/')
shutil.copy(ruta_base + 'Label_Encoder_RFC.pkl', '/content/')
shutil.copy(ruta_base + 'Praevidens.jpg', '/content/')

'/content/Praevidens.jpg'

In [ ]:
%%writefile app.py
import streamlit as st
import numpy as np
import pickle
from PIL import Image

#Cargar imagen
image = Image.open("/content/Praevidens.jpg")

#Diccionario Estados (Código: Nombre)
state_label = {
    'AA': 'Armed Forces Americas',
    'AE': 'Armed Forces Europe',
    'AK': 'Alaska',
    'AL': 'Alabama',
    'AP': 'Armed Forces Pacific',
    'AR': 'Arkansas',
    'AS': 'American Samoa',
    'AZ': 'Arizona',
    'CA': 'California',
    'CO': 'Colorado',
    'CT': 'Connecticut',
    'DC': 'District of Columbia',
    'DE': 'Delaware',
    'FL': 'Florida',
    'GA': 'Georgia',
    'GU': 'Guam',
    'HI': 'Hawaii',
    'IA': 'Iowa',
    'ID': 'Idaho',
    'IL': 'Illinois',
    'IN': 'Indiana',
    'KS': 'Kansas',
    'KY': 'Kentucky',
    'LA': 'Louisiana',
    'MA': 'Massachusetts',
    'MD': 'Maryland',
    'ME': 'Maine',
    'MH': 'Marshall Islands',
    'MI': 'Michigan',
    'MN': 'Minnesota',
    'MO': 'Missouri',
    'MS': 'Mississippi',
    'MT': 'Montana',
    'NC': 'North Carolina',
    'ND': 'North Dakota',
    'NE': 'Nebraska',
    'NH': 'New Hampshire',
    'NJ': 'New Jersey',
    'NM': 'New Mexico',
    'NV': 'Nevada',
    'NY': 'New York',
    'OH': 'Ohio',
    'OK': 'Oklahoma',
    'OR': 'Oregon',
    'PA': 'Pennsylvania',
    'PR': 'Puerto Rico',
    'PW': 'Palau',
    'RI': 'Rhode Island',
    'SC': 'South Carolina',
    'SD': 'South Dakota',
    'TN': 'Tennessee',
    'TX': 'Texas',
    'UT': 'Utah',
    'VA': 'Virginia',
    'VI': 'U.S. Virgin Islands',
    'VT': 'Vermont',
    'WA': 'Washington',
    'WI': 'Wisconsin',
    'WV': 'West Virginia',
    'WY': 'Wyoming'
}

#Códigos state, sector y size
state_name_to_code = {v: k for k, v in state_label.items()}

sector_label = ['Accommodation and Food Services','Administrative and Support and Waste Management and Remediation Services','Agriculture, Forestry, Fishing and Hunting',
               'Arts, Entertainment, and Recreation','Construction','Educational Services','Finance and Insurance','Health Care and Social Assistance','Information',
               'Manufacturing','Management of Companies and Enterprises','Mining, Quarrying, and Oil and Gas Extraction','Other Services','Professional, Scientific, and Technical Services',
               'Public Administration','Real Estate and Rental and Leasing','Retail Trade','Transportation and Warehousing','Unknown','Utilities','Wholesale Trade']

size_dict = {
    "Menos 20 Empleados": 1,
    "20 a 249 Empleados": 2,
    "20 a 99 Empleados": 21,
    "100 a 249 Empleados": 22,
    "Más 250 Empleados": 3,
}

#Cargar Modelos
with open("/content/Modelo_RFR.pkl", "rb") as f:
    modelo_rfr = pickle.load(f)
with open("/content/Modelo_RFC.pkl", "rb") as f:
    modelo_rfc = pickle.load(f)
with open("/content/Label_Encoder_RFR.pkl", "rb") as f:
    le_rfr = pickle.load(f)
with open("/content/Label_Encoder_RFC.pkl", "rb") as f:
    le_rfc = pickle.load(f)

#Diccionarios Mapeo Índices usados en Modelos
state_dict = {k: i for i, k in enumerate(state_label.keys())}
sector_dict = {v: i for i, v in enumerate(sector_label)}

#Diseño App
st.set_page_config(page_title="Praevidens", layout="wide")
st.markdown("<h1 style='color:#1f4e79;'>🔍 Praevidens: Predicción de Riesgo Laboral OSHA</h1>", unsafe_allow_html=True)

col_logo, col_inputs = st.columns([1, 2])

with col_logo:
    st.image(image, use_container_width=True)

with col_inputs:
    c1, c2, c3 = st.columns(3)

    with c1:
        annual_average_employees = st.number_input("👷 Promedio Anual de Empleados", min_value=1)
        total_deaths = st.number_input("💀 Total de Muertes", min_value=0)
        total_other_cases = st.number_input("📋 Otros Casos", min_value=0)

    with c2:
        total_hours_worked = st.number_input("⏱ Total de Horas Trabajadas", min_value=1)
        total_dafw_cases = st.number_input("🏥 Casos DAFW (con Ausencias)", min_value=0)
        total_illnesses = st.number_input("🤒 Total de Enfermedades", min_value=0)

    with c3:
        total_djtr_cases = st.number_input("🧑‍🦽 Casos DJTR (con Restricción)", min_value=0)
        #Mostrar Nombres selectbox ordenados
        selected_state_name = st.selectbox("🌎 Selecciona Estado", sorted(state_name_to_code.keys()))
        selected_state_code = state_name_to_code[selected_state_name]
        sector = st.selectbox("🏭 Selecciona Sector", sector_label)
        size = st.selectbox("🏢 Tamaño Empresa", list(size_dict.keys()))

    st.markdown("---")

    X_input = np.array([[
        annual_average_employees,
        total_hours_worked,
        total_deaths,
        total_dafw_cases,
        total_djtr_cases,
        total_other_cases,
        total_illnesses,
        size_dict[size],
        state_dict[selected_state_code],
        sector_dict[sector]
    ]])

if st.button("🔮 Predecir"):
    risk_index = modelo_rfr.predict(X_input)[0]
    risk_level = modelo_rfc.predict(X_input)[0]

    col1, col2 = st.columns(2)

    with col1:
        st.success(f"📈 Índice de Riesgo: {risk_index:.2f}")

    with col2:
        st.success(f"⚠️ Nivel de Riesgo: {risk_level}")

Writing app.py


In [ ]:
from pyngrok import conf, ngrok

#Authtoken Cuenta Ngrok
conf.get_default().auth_token = "2xcD4MIJX1oK3goufVVXGbubRQv_Eab4DauJGL2GQR2ckk4K"

#Túnel de Streamlit
public_url = ngrok.connect(addr="8501", proto="http")
print(f"Tu app está disponible en: {public_url}")

#Ejecutar Streamlit
!streamlit run app.py &

Tu app está disponible en: NgrokTunnel: "https://b8e3-35-199-1-83.ngrok-free.app" -> "http://localhost:8501"



  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://35.199.1.83:8501

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
